# Convolutional Neural Networks for Sentence Classification

In [37]:
import tensorflow as tf
import numpy as np
import pandas as pd
from collections import Counter
import nltk
import io

### Hyperparameters

> For all datasets we use: rectified linear units, filter
windows $(h)$ of $3, 4, 5$ with $100$ feature maps each,
dropout rate $(p)$ of $0.5$, $l_2$ constraint $(s)$ of $3$, and
mini-batch size of $50$. 



> Training is done through stochastic gradient descent over shuffled mini-batches with the
Adadelta update rule (Zeiler, 2012).


In [38]:
relu = tf.nn.relu
h = [3, 4, 5]
p = 0.5
s = 3
batch_size = 50
Adadelta = tf.train.AdadeltaOptimizer

### Dataset - Movie Reviews

|Data   | c   | l | N  | V  | Vpre | Test
|---|---|---|---|---|
|MR   |  2  | 20  | 10662  | 18765  | 16448 | CV |

In [70]:
def get_data(filename):
    fp = open(filename, 'rb')
    tokenizer = nltk.tokenize.WordPunctTokenizer()
    data = []
    length = 0
    vocab = Counter()
    for line in fp.readlines():
        words = tokenizer.tokenize(line)
        for word in words:
            vocab[word] += 1
        length += len(words)
        data.append(line)
    return data, length, vocab

pos, pos_len, pos_vocab = get_data('data/rt-polarity.pos')
neg, neg_len, neg_vocab = get_data('data/rt-polarity.neg')
data = pos+neg
vocab = pos_vocab + neg_vocab
print("V: " + str(len(vocab)))
print("l: " + str((pos_len+neg_len)/len(data)))
print("N: " + str(len(data)))

V: 18408
l: 22
N: 10662
